# Ratings Prediction- Data Collection

In [2]:
#Importing the required Libraries for web scraping:-
import selenium
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Let's first connect to the web driver
driver=webdriver.Chrome("chromedriver.exe")

In [4]:
#maximizing the window
driver.maximize_window()

# Business Goal:-
We have to scrape at least 20000 rows of data. In this section we need to scrape the reviews of different laptops, Smart Phones, Headphones, smart watches, Professional Cameras, Monitors, Home theater, Router from different e-commerce websites amazon,flipkart etc.

Basically, we need these columns-
1) reviews of the product.
2) rating of the product.

In [5]:
#opening "https://www.amazon.in/"
url="https://www.amazon.in/"
driver.get(url)

In [6]:
#List of products for which reviews has to be collected
products = ["Smart Phone","Laptop", "Headphones", "Smart watch",
            "Professional camera", "Monitor", "Home theater", "Router"]

#list of data values to be collected for
columns = ["Product type", "Title", "Review", "Rating"]

#creating an empty dataframe to which the data scraped will be added
df = pd.DataFrame({} , columns = columns)

In [36]:
#Code for scraping data from amazon

for p in products: #looping
    
    search_box_amazon = driver.find_element_by_id("twotabsearchtextbox") #search box
    search_btn_amazon = driver.find_element_by_id("nav-search-submit-button") #search button
    
    search_box_amazon.clear() #clearing the text... needed from second loop onwards
    search_box_amazon.send_keys(p) #input the keyword to search
    search_btn_amazon.click() #clicking on the search button
    
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") #scrolling to the bottom just to load the entire page
    
    
    for number in range (0,2): # loop for scraping only from 3 pages of search list generated
        search_list = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none']//h2//a//span") #products in search list

        for i in search_list:
            product = i.text
            try:
                i.click() #clicking on the product to open its page
            except:
                continue
            
            driver.switch_to.window(driver.window_handles[-1]) #switching to the product page
            time.sleep(1)
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            try:
                stars = driver.find_element_by_xpath("//table[@id = 'histogramTable']//tr//td[1]//span//a") #ratings element
                stars.click() #proceeding to the ratings page
            except:
                print("No reviews for the product " , product) #if no reviews, helps us to know
    
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            time.sleep(2)
        
            stars_select = driver.find_elements_by_xpath("//table[@id = 'histogramTable']//tr//td[1]//span//a") #list of rating stars
        
            for s in stars_select: #each loop - each star
                s.click() #loading selected star reviews
                time.sleep(1)
            
            
                for num in range(0,2): #reviews in first three pages of selected star
                    
                    reviews_title = driver.find_elements_by_xpath("//div[@data-hook = 'review']//a[@data-hook = 'review-title']") #just knowing the no.of reviews
        
                    for j in range(len(reviews_title)): #helps to scrape individual row and adding to dataframe
                        
                        row = [] #empty row
                        
                        try: #to handle the missing review content
                            
                            #specific title and content for same element
                            reviews_title = driver.find_element_by_xpath("//div[@data-hook = 'review'][{}]//a[@data-hook = 'review-title']".format(j+1))
                            reviews_content =  driver.find_element_by_xpath("//div[@data-hook = 'review'][{}]//span[@data-hook = 'review-body']//span".format(j+1))
                        
                            #creating the data row
                            row.append(p)
                            row.append(reviews_title.text)
                            row.append(reviews_content.text)
                            row.append(s.text)
                        except:
                            continue #skips the next part in loop if anything is missing in the above data row
                            
                        df1 = pd.DataFrame([row] , columns=columns)
                        df = pd.concat([df,df1],ignore_index=True)
                
                    try: #to handle no next page error
                        next_btn  = driver.find_element_by_xpath("//div[@role='navigation']//li[@class='a-last']//a")
                        next_btn.click()
                        time.sleep(2)
                    except:
                        continue #breaks the current loop if there are no more reviews.
        
            driver.close() #closing the reviews page
            driver.switch_to_window(driver.window_handles[0]) #switching to the main page (search list page)
            
            
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        
        #next page of search_list
        next_btn  = driver.find_element_by_xpath("//div[@role='navigation']//ul[@class='a-pagination']//li[@class='a-last']//a")
        next_btn.click()
        time.sleep(1)

No reviews for the product  D-Link R15 AX 1500 Wi-Fi 6 AI Router, 4 Gigabit Ports, 4 External Antenna, Voice Control, Parental Control, D-Link Mesh Support
No reviews for the product  D-Link DIR-X5460 WiFi 6 Router AX5400 MU-MIMO Voice Control Compatible with Alexa & Google Assistant, Dual Band Gigabit Gaming Internet Network, Black
No reviews for the product  D-Link R15 AX 1500 Wi-Fi 6 AI Router, 4 Gigabit Ports, 4 External Antenna, Voice Control, Parental Control, D-Link Mesh Support
No reviews for the product  2300mAh OriginaI WD670 Router Battery for ZTE wd670 Jiofi m2 / Jiofi m2s Jiofi 2 Router Airtel 4g Hotspot Router Jio fi2 Wireless Hotspot Router with 3 Months Warranty


In [38]:
df11=pd.read_csv("Rating1.csv")
df11

,Unnamed: 0,Product type,Title,Review,Rating
0,0,Smart Phone,Best phone but ( Prime member got 1000 loss),"Pros (exellent disply and its very smooth, and...",5 star
1,1,Smart Phone,"Super slim and smooth, I'm love in it.","Got it first by prime...smooth and slim, batte...",5 star
2,2,Smart Phone,after 1 Day of usage,A day spent on exploring my new device and I w...,5 star
3,3,Smart Phone,Very very sleek I didn’t expect that slim,I didn’t expect this kind of slimness of the p...,5 star
4,4,Smart Phone,Samsung Top Quality budget phone,Ordered on 2nd Oct. Received on 3rd Oct/Aamzon...,5 star
...,...,...,...,...,...
18630,18630,Home theater,Very nice experience !,I like the overall sound quality and good bass...,5 star
18631,18631,Home theater,Fabulous home theater,"Wonderful product by I kall , sound quality is...",5 star
18632,18632,Home theater,One of the best home theater,Sound clarity is best and it is good for your ...,5 star
18633,18633,Home theater,Worst ever products,I buy this home theatre.it performance is very...,1 star


In [39]:
df12=pd.read_csv("Rating2.csv")
df12

,Unnamed: 0,Product type,Title,Review,Rating
0,0,Router,Good one for Cable Broadband,I am using Alliance Broadband in Chuchura. Ini...,5 star
1,1,Router,No issues But range not much better than DIR 600M,I didn't have any issues with the connection. ...,5 star
2,2,Router,A real gem for your home/small office wifi need,A real gem for your home/small office wifi nee...,5 star
3,3,Router,--------Awesome Product---------- Easy to Conf...,--------Awesome Product----------\nEasy to Con...,5 star
4,4,Router,Good quality stuff,Good quality stuff. We use ACT fibre net. We h...,5 star
...,...,...,...,...,...
3287,3287,Router,"Cute device, worked fine for me, recommended","Need to check backup, using 9v output.",5 star
3288,3288,Router,Best UPS for Wifi Routers,I have been searching for a router ups as I fa...,5 star
3289,3289,Router,Working as expected,Very easy setup. Packing need to improve. At ...,5 star
3290,3290,Router,Hold good charge and powers well,Works very well and holds good power. I had a ...,5 star


In [40]:
#Merging both dataframes
rating_data=pd.merge(df11,df12,how='outer')
rating_data

,Unnamed: 0,Product type,Title,Review,Rating
0,0,Smart Phone,Best phone but ( Prime member got 1000 loss),"Pros (exellent disply and its very smooth, and...",5 star
1,1,Smart Phone,"Super slim and smooth, I'm love in it.","Got it first by prime...smooth and slim, batte...",5 star
2,2,Smart Phone,after 1 Day of usage,A day spent on exploring my new device and I w...,5 star
3,3,Smart Phone,Very very sleek I didn’t expect that slim,I didn’t expect this kind of slimness of the p...,5 star
4,4,Smart Phone,Samsung Top Quality budget phone,Ordered on 2nd Oct. Received on 3rd Oct/Aamzon...,5 star
...,...,...,...,...,...
21922,3287,Router,"Cute device, worked fine for me, recommended","Need to check backup, using 9v output.",5 star
21923,3288,Router,Best UPS for Wifi Routers,I have been searching for a router ups as I fa...,5 star
21924,3289,Router,Working as expected,Very easy setup. Packing need to improve. At ...,5 star
21925,3290,Router,Hold good charge and powers well,Works very well and holds good power. I had a ...,5 star


In [41]:
#checking rating numbers detail
rating_data.Rating.value_counts()

5 star    4960
1 star    4534
4 star    4497
3 star    4117
2 star    3819
Name: Rating, dtype: int64

In [42]:
#checking missing values
rating_data.isnull().sum()

Unnamed: 0      0
Product type    0
Title           1
Review          1
Rating          0
dtype: int64

In [43]:
rating_data[rating_data.isna().any(axis=1)]

,Unnamed: 0,Product type,Title,Review,Rating
3952,3952,Laptop,NaN,NaN,4 star


In [44]:
rating_data = rating_data.fillna("Blank")

In [45]:
#checking missing values again
rating_data.isnull().sum()

Unnamed: 0      0
Product type    0
Title           0
Review          0
Rating          0
dtype: int64

In [46]:
#saving cardata as csv file
rating_data.to_csv("rating_data.csv")

# || Happy Ending||